# RT2 Assignment2

- Create a jupyter notebook to replace the user interface (the node «A»)
- Use widgets to let the user know the position of the robot and all targets that have been set and cancelled in the environment and the distance of the closest obstacle

In [1]:
#import jupyter and graphs libraries
%matplotlib widget
import jupyros as jr
import ipywidgets as widgets
import numpy as np
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from sensor_msgs.msg import LaserScan
from IPython.display import display
from ipywidgets import FloatText, Layout, HBox

#import ROS-related libraries and messages
import rospy
import actionlib
import actionlib.msg

import assignment_2_2022
import assignment_2_2022.msg
from std_srvs.srv import *
from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
from assignment_2_2022.msg import InfoMsg

In [2]:
#initialize goal
goal = None

Node initialization:

In [3]:
#initialize the ROS node
rospy.init_node('jupyter_nodeA')

#create a custom msg publisher on the '/robot_info' topic
pub = rospy.Publisher('/robot_info', InfoMsg, queue_size=1)

#creates the action client
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)

#waits for the server to be ready
client.wait_for_server()

True

Widget creation:

In [4]:
#create bounded float text widgets for goal coordinates
x_goal = widgets.BoundedFloatText(value = 0.0, description = 'x', min = -9.0, max = 9.0,
                            style = {'description_width': 'initial'}, step = 0.1, 
                            layout = widgets.Layout(width = '100px'))

y_goal = widgets.BoundedFloatText(value = 0.0, description = 'y', min = -9.0, max = 9.0,
                            style = {'description_width': 'initial'}, step = 0.1, 
                            layout = widgets.Layout(width = '100px'))

#create button widgets for sending and canceling goals
send_goal = widgets.Button(
    value=False,
    description='Send goal',
    disabled=False,
    button_style='success',
)

cancel_goal = widgets.Button(
    value=False,
    description='Cancel goal',
    disabled=False,
    button_style='danger',
)

#create a float text widget to display the distance from the closest obstacle
obs_distance = widgets.FloatText(value=0.0, description='Obstacle Distance:', disabled=True)
obs_distance.layout = Layout(width='auto')

Definition of the callback function for the '/odom' topic:

In [5]:
def callback(msg):

    global pub
    
    #get position and linear velocity from msg
    position = msg.pose.pose.position
    velocity = msg.twist.twist.linear
    
    #create custom msg
    robot_info = InfoMsg()
    robot_info.x = position.x
    robot_info.y = position.y
    robot_info.velX = velocity.x
    robot_info.velY = velocity.y

    #publish robot_info
    pub.publish(robot_info)

Defininition of the callback function for the '/scan' topic

In [6]:
def laser_callback(msg):
    
    #get range values from the laser scan msg   
    ranges = msg.ranges
    
    #get the closest obstacle by taking the minimum value inside the list 
    nearest_obstacle_range = min(ranges)
            
    #set the output with the previously found minimum
    obs_distance.value=nearest_obstacle_range

Buttons actions:

In [7]:
def sendGoal(btn):
    
    global goal
    
    #set the goal position with the previously entered coordinates
    goal = assignment_2_2022.msg.PlanningGoal()
    goal.target_pose.pose.position.x = x_goal.value
    goal.target_pose.pose.position.y = y_goal.value
    
    #send the goal to the action server
    client.send_goal(goal)
    
#activate the send button when clicked    
send_goal.on_click(sendGoal)

In [8]:
def cancelGoal(btn):
    
    global goal
    
    #cancel the active
    if client.get_state() == actionlib.GoalStatus.ACTIVE:
        goal = None
        client.cancel_goal()

#activate the cancel button when clicked            
cancel_goal.on_click(cancelGoal)

Definition of the class responsible for the visualization of the reached/cancelled goals using a bar graph:

In [9]:
class Goal_Visualizer:
    
    def __init__(self):
        
        #initialize the figure
        self.fig, self.ax_bar = plt.subplots()

        #initialize the counters relative to reached and cancelled goals
        self.reached = 0
        self.cancelled = 0

        #set the title and the colors of the graph
        self.ax_bar.set_title('Goal Statistics')
        self.bar_colors = ['green', 'red']

        #set the labels and positions for the bars on the x-axis
        self.bar_labels = ('Reached', 'Cancelled')
        self.bar_pos = np.arange(len(self.bar_labels))
        self.ax_bar.set_xticks(self.bar_pos)
        self.ax_bar.set_xticklabels(self.bar_labels)

        #set the y-axis limits and ticks
        self.ax_bar.set_ylim([0, 3])
        self.ax_bar.set_yticks(np.arange(0, 3, 1))

        #create the initial bar plot with the counters set to 0
        self.bar_plot = self.ax_bar.bar(self.bar_pos, [self.reached, self.cancelled], align='center', color=self.bar_colors)

    def goal_callback(self, msg):
        
        #check the status and update the two counters based on the result
        if msg.status.status == 3:
            self.reached += 1
        elif msg.status.status == 2:
            self.cancelled += 1

    def update(self, frame):
        
        self.bar_counts = [self.reached, self.cancelled]

        #update the bar heights based on the updated counters
        for i, bar in enumerate(self.bar_plot):
            if i == 0:
                bar.set_height(self.reached)
            else:
                bar.set_height(self.cancelled)

        #adjust the y-axis limits and ticks based on the maximum counter value
        self.ax_bar.set_ylim([0, max(self.bar_counts) + 2])
        self.ax_bar.set_yticks(np.arange(0, max(self.bar_counts) + 2, 1))

        return self.bar_plot


Definition of the class responsible for the visualization of the goal position and the robot trajectory:

In [10]:
class Trajectory_Visualizer:
    
    def __init__(self):
        
        #initialize the figure
        self.fig, self.ax_trajectory = plt.subplots()

        #initialize the arrays relative to the robot's trajectory and the position of the target
        self.robot_x = [0]
        self.robot_y = [0] #initialized to (0,0)
        self.target_x = []
        self.target_y = []
        
        #initialize the robot's trajectory and the goal's marker
        self.robot_line, = self.ax_trajectory.plot([], [], color='red', linewidth=5)
        self.goal_marker, = self.ax_trajectory.plot([], [], marker='*', color='yellow', markersize=8)

        #set title and axis
        self.ax_trajectory.set_title('Robot Trajectory')
        self.ax_trajectory.set_xlabel("X")
        self.ax_trajectory.set_ylabel("Y")
        self.ax_trajectory.set_xlim(-10, 10)
        self.ax_trajectory.set_ylim(-10, 10)
        self.ax_trajectory.set_aspect('equal')
        self.ax_trajectory.grid(True, linestyle='--', linewidth=0.5)

    def odom_callback(self, msg):
        
        #update the array for the robot trajectory 
        self.robot_x.append(msg.pose.pose.position.x) 
        self.robot_y.append(msg.pose.pose.position.y)

    def update(self, frame):
        
        self.robot_line.set_data(self.robot_x, self.robot_y)

        #update the goal marker position
        if goal is not None:
            self.target_x = goal.target_pose.pose.position.x
            self.target_y = goal.target_pose.pose.position.y
        else:
            self.target_x = None
            self.target_y = None

        self.goal_marker.set_data(self.target_x, self.target_y)

# MAIN

Set the goal position or cancell the active one:

In [11]:
display(widgets.HBox([x_goal, y_goal]))
display(widgets.HBox([send_goal, cancel_goal]))

Display the distance from the closest obstacle:

In [12]:
display(HBox([obs_distance]))

Subscribe to the topics '/odom' and 'scan' to get the current robot position and the datas picked up from the laser:

In [13]:
jr.subscribe('/odom', Odometry, callback)
jr.subscribe('/scan', LaserScan, laser_callback)

Display the bar graph relative to the reached/cancelled goals:

In [14]:
bar = Goal_Visualizer()
results = rospy.Subscriber('/reaching_goal/result', assignment_2_2022.msg.PlanningActionResult, bar.goal_callback)

animation_bar = FuncAnimation(bar.fig, bar.update, interval = 1000)
plt.show(block = True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Display the plot relative to the goal position and the robot trajectory:

In [15]:
traj = Trajectory_Visualizer()
sub = rospy.Subscriber('/odom', Odometry, traj.odom_callback)

animation_traj = FuncAnimation(traj.fig, traj.update, interval = 1000)
plt.show(block = True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …